In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import KFold, StratifiedKFold

In [3]:
df_org = pd.read_csv("train.csv", encoding="cp949")

In [4]:
df_org.head()

,Unnamed: 0,신고번호,신고일자,통관지세관부호,신고인부호,수입자부호,해외거래처부호,특송업체부호,수입통관계획코드,수입신고구분코드,...,운송수단유형코드,반입보세구역부호,HS10단위부호,적출국가코드,원산지국가코드,관세율구분코드,관세율,검사결과코드,우범여부,핵심적발
0,0,57298928,2020-01-01,121,2O5A2,82ZHWNL,NaN,TQ18AK,D,B,...,10,2106003,8481900000,US,US,A,8.0,N3,1,1
1,1,85092852,2020-01-01,30,305K5,5IS70LE,NaN,NaN,C,B,...,40,4077010,2106909099,US,US,A,8.0,A,0,0
2,2,63014158,2020-01-01,20,CGMT6,GJ5KBL3,R9ZQOG7,NaN,D,B,...,40,4077007,6307909000,US,US,A,10.0,A,0,0
3,3,40175917,2020-01-01,40,QWUTG,PBYW02T,NaN,NaN,C,B,...,40,4077106,6505009090,CN,CN,A,8.0,A,0,0
4,4,11602631,2020-01-01,30,0X1CO,MCX0GJB,4Z9PX0Y,NaN,C,B,...,40,2006075,6204320000,CN,CN,FCN1,5.2,M1_N1,1,1


# 범주형 변수 수치화

In [5]:
df_org = df_org.fillna('없음')

In [6]:
df_z = df_org['특송업체부호'].value_counts(normalize=True).rename_axis('unique').reset_index(name='counts')
df_un = df_z['unique']
df_co = df_z['counts'].round(decimals=6)
du = df_un.tolist()
dc = df_co.tolist()
for i in range(df_z.shape[0]):
    df_org.loc[df_org['특송업체부호'] == du[i] , '특송업체부호비율'] = dc[i]
df_org['특송업체부호비율'] = df_org['특송업체부호비율'].apply(lambda x: np.log1p(x))

In [7]:
df_z = df_org['통관지세관부호'].value_counts(normalize=True).rename_axis('unique').reset_index(name='counts')
df_un = df_z['unique']
df_co = df_z['counts'].round(decimals=6)
du = df_un.tolist()
dc = df_co.tolist()
for i in range(df_z.shape[0]):
    df_org.loc[df_org['통관지세관부호'] == du[i] , '통관지세관부호비율'] = dc[i]
df_org['통관지세관부호비율'] = df_org['통관지세관부호비율'].apply(lambda x: np.log1p(x))

In [8]:
df_z = df_org['해외거래처부호'].value_counts(normalize=True).rename_axis('unique').reset_index(name='counts')
df_un = df_z['unique']
df_co = df_z['counts'].round(decimals=6)
du = df_un.tolist()
dc = df_co.tolist()
for i in range(df_z.shape[0]):
    df_org.loc[df_org['해외거래처부호'] == du[i] , '해외거래처부호비율'] = dc[i]
df_org['해외거래처부호비율'] = df_org['해외거래처부호비율'].apply(lambda x: np.log1p(x))

In [9]:
df_z = df_org['운송수단유형코드'].value_counts(normalize=True).rename_axis('unique').reset_index(name='counts')
df_un = df_z['unique']
df_co = df_z['counts'].round(decimals=6)
du = df_un.tolist()
dc = df_co.tolist()
for i in range(df_z.shape[0]):
    df_org.loc[df_org['운송수단유형코드'] == du[i] , '운송수단유형코드비율'] = dc[i]
df_org['운송수단유형코드비율'] = df_org['운송수단유형코드비율'].apply(lambda x: np.log1p(x))

In [10]:
df_org.head(5)

,Unnamed: 0,신고번호,신고일자,통관지세관부호,신고인부호,수입자부호,해외거래처부호,특송업체부호,수입통관계획코드,수입신고구분코드,...,원산지국가코드,관세율구분코드,관세율,검사결과코드,우범여부,핵심적발,특송업체부호비율,통관지세관부호비율,해외거래처부호비율,운송수단유형코드비율
0,0,57298928,2020-01-01,121,2O5A2,82ZHWNL,없음,TQ18AK,D,B,...,US,A,8.0,N3,1,1,0.014612,0.001781,0.294304,0.383327
1,1,85092852,2020-01-01,30,305K5,5IS70LE,없음,없음,C,B,...,US,A,8.0,A,0,0,0.514771,0.111346,0.294304,0.393836
2,2,63014158,2020-01-01,20,CGMT6,GJ5KBL3,R9ZQOG7,없음,D,B,...,US,A,10.0,A,0,0,0.514771,0.149094,0.000104,0.393836
3,3,40175917,2020-01-01,40,QWUTG,PBYW02T,없음,없음,C,B,...,CN,A,8.0,A,0,0,0.514771,0.410558,0.294304,0.393836
4,4,11602631,2020-01-01,30,0X1CO,MCX0GJB,4Z9PX0Y,없음,C,B,...,CN,FCN1,5.2,M1_N1,1,1,0.514771,0.111346,0.000065,0.393836


# 변수 Drop

In [ ]:
del df_org['통관지세관부호']
del df_org['특송업체부호']
del df_org['해외거래처부호']
del df_org['운송수단유형코드']
del df_org['신고일자']
del df_org['검사결과코드']
del df_org['핵심적발']
del df_org['신고인부호']
del df_org['신고번호']
del df_org['원산지국가코드']
del df_org['관세율']

In [ ]:
df_org.columns

In [ ]:
# 범주형 변수 지정
discrete_columns = ['수입자부호', '수입통관계획코드', 
       '수입신고구분코드', '수입거래구분코드',
       '수입종류코드', '징수형태코드', 
       '반입보세구역부호', 
       'HS10단위부호', '적출국가코드', '관세율구분코드']

# 범주형 변수들을 문자열로 지정 ('object' -> 'string')
for var in discrete_columns:
    df_org[var] = df_org[var].astype(str)

# 레이블 인코딩

In [ ]:
# 범주형 변수 전처리 1단계: label encoding --> 각 범주형 변수가 갖는 클래스에 고유의 식별번호 부여
# 예) 수입자 상호: AAABB -> 1, 가나다라 -> 2, ...

label_encoding_ref = {}
for var in discrete_columns:
    label_encoding_ref[var] = {code: i+1 for i, code in enumerate(df_org[var].unique())}
    print(label_encoding_ref[var])
    df_org[var] = [label_encoding_ref[var][x] for x in df_org[var]]

In [ ]:
df_org.head()

In [ ]:
# 수치형 변수 지정
numeric_columns = ['신고중량(KG)', '과세가격원화금액', '특송업체부호비율', 
                   '통관지세관부호비율', '해외거래처부호비율', '운송수단유형코드비율']

In [ ]:
# 수치형 변수, 범주형 변수 분리
numeric = df_org[numeric_columns]

In [ ]:
# 수치형 변수 정규화
from sklearn.preprocessing import StandardScaler, RobustScaler
std_scaler = StandardScaler()
rob_scaler = RobustScaler()

numeric['신고중량(KG)'] = rob_scaler.fit_transform(numeric['신고중량(KG)'].values.reshape(-1,1))
numeric['과세가격원화금액'] = rob_scaler.fit_transform(numeric['과세가격원화금액'].values.reshape(-1,1))
numeric['특송업체부호비율'] = rob_scaler.fit_transform(numeric['특송업체부호비율'].values.reshape(-1,1))
numeric['통관지세관부호비율'] = rob_scaler.fit_transform(numeric['통관지세관부호비율'].values.reshape(-1,1))
numeric['해외거래처부호비율'] = rob_scaler.fit_transform(numeric['해외거래처부호비율'].values.reshape(-1,1))
numeric['운송수단유형코드비율'] = rob_scaler.fit_transform(numeric['운송수단유형코드비율'].values.reshape(-1,1))

In [ ]:
df_org.head()

# 수치형 PCA

In [ ]:
from sklearn import decomposition
pca = decomposition.PCA(n_components=5).fit(numeric)

reduced_numeric = pca.transform(numeric)

In [ ]:
pca.explained_variance_ratio_

In [ ]:
reduced_numeric.shape

In [ ]:
df_org['신고중량(KG)']  = reduced_numeric[:, 0]
df_org['과세가격원화금액']  = reduced_numeric[:, 1]
df_org['특송업체부호비율']  = reduced_numeric[:, 2]
df_org['통관지세관부호비율']  = reduced_numeric[:, 2]
df_org['해외거래처부호비율']  = reduced_numeric[:, 3]
df_org['운송수단유형코드비율']  = reduced_numeric[:, 4]

# 데이터셋 확인

In [ ]:
df_org.head(5)

# 훈련/시험 데이터셋 분할

In [ ]:
y = df_org['우범여부']
x = df_org.drop(columns = '우범여부')
type(x), type(y)

In [ ]:
df_org_train, df_org_test, org_train_y, org_test_y = train_test_split(x,y, test_size = 0.3, stratify = y, random_state =3)

# 원핫인코딩 데이터셋 프로세스

In [ ]:
df_org_train['label'] = 'train'
df_org_test['label'] = 'test'

concat_df = pd.concat([df_org_train,df_org_test])

df_org_trains = pd.get_dummies(concat_df, columns = discrete_columns)

df_train =  df_org_trains[df_org_trains['label'] == 'train']
df_test =  df_org_trains[df_org_trains['label'] == 'test']

In [ ]:
df_train = df_train.drop('label', axis=1)
df_test = df_test.drop('label', axis=1)

In [ ]:
# 데이터 사이즈 확인
print(df_train.shape, df_test.shape)
print(org_train_y.shape, org_test_y.shape)

# Imbalanced Sampling
# * SMOTE *

In [ ]:
from imblearn.over_sampling import *

In [ ]:
df_train.info()

In [ ]:
smote = SMOTE(random_state=11, sampling_strategy='all')
df_train_over_x, df_train_over_y = smote.fit_resample(df_train, org_train_y)
df_train_over_x.info(), df_train_over_y

# 모델 생성 / 학습 / 평가

In [ ]:
# XGBoost 모델 생성
from xgboost import XGBClassifier
from sklearn.metrics import f1_score,roc_auc_score
import warnings
warnings.filterwarnings("ignore")

In [ ]:
xgb_clf = XGBClassifier(n_estimators=50, max_depth=4, n_jobs=-1, reg_lambda=1)

eval_set = [(df_train_over_x, df_train_over_y), (df_test, org_test_y)]

xgb_clf.fit(df_train_over_x, df_train_over_y, eval_metric=["logloss"],
            eval_set=eval_set, verbose=True)

In [ ]:
# 모델 성능 시각화

from matplotlib import pyplot
# retrieve performance metrics
results = xgb_clf.evals_result()
epochs = len(results['validation_0']['logloss'])
x_axis = range(0, epochs)
# plot log loss
fig, ax = pyplot.subplots()
ax.plot(x_axis, results['validation_0']['logloss'], label='Train')
ax.plot(x_axis, results['validation_1']['logloss'], label='Test')
ax.legend()
pyplot.ylabel('Log Loss')
pyplot.title('XGBoost Log Loss')

In [ ]:
# evaluate xgboost model
print("------Evaluating xgboost model------")
# Predict
test_pred = xgb_clf.predict_proba(df_test)[:,1]
# Calculate auc
xgb_auc = roc_auc_score(org_test_y, test_pred)
print(xgb_auc)

In [ ]:
def inspection_performance(predicted_fraud, test_fraud):
        
    Inspect_Rate=[]
    Precision=[]
    Recall=[]
    
    for i in range(0,101,1):
        
        threshold = np.percentile(predicted_fraud, i)
        # Precision = number of frauds / number of inspection
        precision = np.mean(test_fraud[predicted_fraud >= threshold])
        # Recall = number of inspected frauds / number of frauds
        recall = sum(test_fraud[predicted_fraud >= threshold])/sum(test_fraud)
        # Save values
        Inspect_Rate.append(100-i)
        Precision.append(precision)
        Recall.append(recall)
        
    
    compiled_conf_matrix = pd.DataFrame({
        'Inspect_Rate':Inspect_Rate,
        'Precision':Precision,
        'Recall':Recall
    })

    return compiled_conf_matrix

In [ ]:
basic_performance = inspection_performance(test_pred, org_test_y.astype(float))

In [ ]:
# 검사율 1~10% 지정 시 Precision 및 Recall 분석
# Precision (적중률) = (검사선별된 우범건수)/(검사선별 건수)
# Recall (적발률) = (검사선별된 우범건수) / (전체 우범건수)
basic_performance.iloc[range(99,89,-1),:]

In [ ]:
data = pd.melt(basic_performance, 
               id_vars = ['Inspect_Rate'],
               value_vars = ['Recall','Precision'])

sns.relplot(data=data,
            kind='line',
            x="Inspect_Rate", 
            y="value", 
            hue='variable',
            col="variable")

In [ ]:
# 모델에 기여도가 높은 변수 시각화
from xgboost import plot_importance
plt.rcParams["font.family"] = 'Malgun Gothic'
plt.rcParams["figure.figsize"] = (15,10)
plot_importance(xgb_clf, max_num_features=30)
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
y_preds = xgb_clf.predict(df_test)

In [ ]:
print("accuracy score for trained data",accuracy_score(df_train_over_y,xgb_clf.predict(df_train_over_x)))
print("accuracy score is",accuracy_score(org_test_y,y_preds))
print("Confusion matrix",confusion_matrix(org_test_y,y_preds))
print("Report",classification_report(org_test_y,y_preds))

# Submission 값 추출

In [ ]:
## test 파일 전처리

In [ ]:
df_want = pd.read_csv("test.csv", encoding="cp949")

In [ ]:
df_org = df_want.fillna('없음')

In [ ]:
df_z = df_org['특송업체부호'].value_counts(normalize=True).rename_axis('unique').reset_index(name='counts')
df_un = df_z['unique']
df_co = df_z['counts'].round(decimals=6)
du = df_un.tolist()
dc = df_co.tolist()
for i in range(df_z.shape[0]):
    df_org.loc[df_org['특송업체부호'] == du[i] , '특송업체부호비율'] = dc[i]
df_org['특송업체부호비율'] = df_org['특송업체부호비율'].apply(lambda x: np.log1p(x))

In [ ]:
df_z = df_org['통관지세관부호'].value_counts(normalize=True).rename_axis('unique').reset_index(name='counts')
df_un = df_z['unique']
df_co = df_z['counts'].round(decimals=6)
du = df_un.tolist()
dc = df_co.tolist()
for i in range(df_z.shape[0]):
    df_org.loc[df_org['통관지세관부호'] == du[i] , '통관지세관부호비율'] = dc[i]
df_org['통관지세관부호비율'] = df_org['통관지세관부호비율'].apply(lambda x: np.log1p(x))

In [ ]:
df_z = df_org['해외거래처부호'].value_counts(normalize=True).rename_axis('unique').reset_index(name='counts')
df_un = df_z['unique']
df_co = df_z['counts'].round(decimals=6)
du = df_un.tolist()
dc = df_co.tolist()
for i in range(df_z.shape[0]):
    df_org.loc[df_org['해외거래처부호'] == du[i] , '해외거래처부호비율'] = dc[i]
df_org['해외거래처부호비율'] = df_org['해외거래처부호비율'].apply(lambda x: np.log1p(x))

In [ ]:
df_z = df_org['운송수단유형코드'].value_counts(normalize=True).rename_axis('unique').reset_index(name='counts')
df_un = df_z['unique']
df_co = df_z['counts'].round(decimals=6)
du = df_un.tolist()
dc = df_co.tolist()
for i in range(df_z.shape[0]):
    df_org.loc[df_org['운송수단유형코드'] == du[i] , '운송수단유형코드비율'] = dc[i]
df_org['운송수단유형코드비율'] = df_org['운송수단유형코드비율'].apply(lambda x: np.log1p(x))

In [ ]:
del df_org['통관지세관부호']
del df_org['특송업체부호']
del df_org['해외거래처부호']
del df_org['운송수단유형코드']
del df_org['신고일자']
del df_org['검사결과코드']
del df_org['핵심적발']
del df_org['신고인부호']
del df_org['신고번호']
del df_org['원산지국가코드']
del df_org['관세율']
df_org.head(5)

In [ ]:
# 범주형 변수 지정
discrete_columns = ['수입자부호', '수입통관계획코드', 
       '수입신고구분코드', '수입거래구분코드',
       '수입종류코드', '징수형태코드', 
       '반입보세구역부호', 
       'HS10단위부호', '적출국가코드', '관세율구분코드']

# 범주형 변수들을 문자열로 지정 ('object' -> 'string')
for var in discrete_columns:
    df_org[var] = df_org[var].astype(str)

label_encoding_ref = {}
for var in discrete_columns:
    label_encoding_ref[var] = {code: i+1 for i, code in enumerate(df_org[var].unique())}
    print(label_encoding_ref[var])
    df_org[var] = [label_encoding_ref[var][x] for x in df_org[var]]

In [ ]:
# 수치형 변수 지정
numeric_columns = ['신고중량(KG)', '과세가격원화금액', '특송업체부호비율', 
                   '통관지세관부호비율', '해외거래처부호비율', '운송수단유형코드비율']
# 수치형 변수, 범주형 변수 분리
numeric = df_org[numeric_columns]
# 수치형 변수 정규화
from sklearn.preprocessing import StandardScaler, RobustScaler
std_scaler = StandardScaler()
rob_scaler = RobustScaler()

numeric['신고중량(KG)'] = rob_scaler.fit_transform(numeric['신고중량(KG)'].values.reshape(-1,1))
numeric['과세가격원화금액'] = rob_scaler.fit_transform(numeric['과세가격원화금액'].values.reshape(-1,1))
numeric['특송업체부호비율'] = rob_scaler.fit_transform(numeric['특송업체부호비율'].values.reshape(-1,1))
numeric['통관지세관부호비율'] = rob_scaler.fit_transform(numeric['통관지세관부호비율'].values.reshape(-1,1))
numeric['해외거래처부호비율'] = rob_scaler.fit_transform(numeric['해외거래처부호비율'].values.reshape(-1,1))
numeric['운송수단유형코드비율'] = rob_scaler.fit_transform(numeric['운송수단유형코드비율'].values.reshape(-1,1))

In [ ]:
pca = decomposition.PCA(n_components=5).fit(numeric)

reduced_numeric = pca.transform(numeric)
pca.explained_variance_ratio_

df_org['신고중량(KG)']  = reduced_numeric[:, 0]
df_org['과세가격원화금액']  = reduced_numeric[:, 1]
df_org['특송업체부호비율']  = reduced_numeric[:, 2]
df_org['통관지세관부호비율']  = reduced_numeric[:, 2]
df_org['해외거래처부호비율']  = reduced_numeric[:, 3]
df_org['운송수단유형코드비율']  = reduced_numeric[:, 4]

In [ ]:
df_org.head(5)

In [ ]:
df_test = df_org
y_preds = xgb_clf.predict(df_test)

In [ ]:
y_preds

In [ ]:
pred_df = pd.DataFrame(y_preds, columns=['우범여부'])
pred_df

In [ ]:
pred_df.to_csv('C:\\Users\\CEO\\result.csv', index = None)